In [1]:
import os
from azureml.core import Model, Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice, AksWebservice
from azureml.pipeline.wrapper import PipelineRun

In [2]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


In [3]:
# get a PipelineRun object
experiment_name = "deploy"
experiment = Experiment(workspace, experiment_name)
run_id = '24f0ba77-1ef8-4a9a-b9a8-9ed14d98190e'
pipeline_run = PipelineRun(experiment, run_id)

In [4]:
# get a StepRun object
step_run = pipeline_run.find_step_run(name='FastText Train')[0]

In [5]:
# download model from the output port of the Train module
port = step_run.get_port(name='Trained model dir')
saved_path = port.download(overwrite=True)
print('model save at: {}'.format(saved_path))

C:\Users\50657\miniconda3\envs\tmp2\lib\site-packages\azureml\pipeline\wrapper\_pipeline_run.py:699: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  yaml_dict = yaml.load(yaml_str)


Downloaded azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir/BestModel, 1 files out of an estimated total of 1
model save at: data\azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir


In [6]:
# register model for deployment
model_name = os.listdir(saved_path)[0]
model_path = os.path.join(saved_path, model_name)
model = Model.register(workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy': 1})

Registering model model_for_deploy


In [7]:
env_list = Environment.list(workspace)
name = 'env_for_deployment'
if name not in env_list:
    file_path = 'deployment/env_for_deployment.yaml'
    env = Environment.from_conda_specification(name=name, file_path=file_path)
    env = env.register(workspace=workspace)
else:
    env = Environment.get(workspace=workspace, name=name)

In [8]:
# define inference configuration
entry_script = 'scoring_for_deployment.py'
source_directory = 'deployment'
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                   environment=env)

In [ ]:
# deploy locally
service_name = 'local-deploy-test'
models = [model]
port = 8892
deployment_config = LocalWebservice.deploy_configuration(port=port)
service_locally = Model.deploy(workspace=workspace, name=service_name, models=models, inference_config=inference_config,
                               deployment_config=deployment_config)
service_locally.wait_for_deployment(show_output=True)
print(service_locally.state)
print(service_locally.get_logs())

In [14]:
# deploy to ACI (Azure Container Instances)
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand.
# to delete the existing service, let overwrite be True
service_name = 'aci-deploy-test'
models = [model]
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_aci = Model.deploy(workspace, service_name, models=models, inference_config=inference_config,
                           deployment_config=deployment_config, overwrite=True)
service_aci.wait_for_deployment(show_output=True)
print(service_aci.state)

Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
# deploy to AKS (Azure Kubernetes Service)
# workspace with AKS
aks_subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
aks_resource_group = 'DesignerDRI'
aks_workspace_name = 'DesignerDRI_EASTUS'
aks_workspace = Workspace(aks_subscription_id, aks_resource_group, aks_workspace_name)

In [10]:
# deploy to AKS (Azure Kubernetes Service)
# register model in this workspace
model = Model.register(aks_workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy': 1})

Registering model model_for_deploy


In [19]:
# deploy to AKS (Azure Kubernetes Service)
# load aks_target
name = 'attached-aks'
compute_target_name_list = [target.name for target in AksCompute.list(aks_workspace)]
if not name in compute_target_name_list:
    aks_resource_group = 'AmlStudioV2DRI'
    cluster_name = 'aks-dev-6node33512023a'
    attach_config = AksCompute.attach_configuration(resource_group=aks_resource_group, cluster_name=cluster_name)
    aks_target = ComputeTarget.attach(aks_workspace, name, attach_config)
    aks_target.wait_for_completion(show_output=True)
else:
    aks_target = AksCompute(aks_workspace, name)
aks_target = AksCompute(aks_workspace, name)
aks_target

Creating............................
SucceededProvisioning operation finished, operation "Succeeded"


AksCompute(workspace=Workspace.create(name='DesignerDRI_EASTUS', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='DesignerDRI'), name=attached-aks, id=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourceGroups/DesignerDRI/providers/Microsoft.MachineLearningServices/workspaces/DesignerDRI_EASTUS/computes/attached-aks, type=AKS, provisioning_state=Succeeded, location=eastus, tags=None)

In [12]:
sss = aks_workspace.webservices


In [13]:
sss['aks-deploy-test'].delete()

In [14]:
sss = aks_workspace.webservices

In [15]:
# deploy to AKS (Azure Kubernetes Service)
# every time we deploy to AKS, we need to change the service_name or we delete the existing service beforehand
service_name = 'aks-deploy-test'
models = [model]
token_auth_enabled = True
# Only one type of Auth may be enabled
if token_auth_enabled:
    # key auth
    auth_enabled = False
else:
    auth_enabled = True

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                       token_auth_enabled=token_auth_enabled,
                                                       auth_enabled=auth_enabled)
service_aks = Model.deploy(aks_workspace, service_name, models, inference_config, deployment_config, aks_target,
                       overwrite=True)
service_aks.wait_for_deployment(show_output=True)
print(service_aks.state)

Running........
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [18]:
# consume
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type': 'application/json'})
# print(headers)


standard_sample_input = {'param': {'input_sentence': '受疫情影响, 今年很多学生不得不在家上课'}}
standard_sample_input = json.dumps(standard_sample_input)

# service = service_locally
# service = service_aci

service = service_aks
token, refresh_by = service.get_token()
headers['Authorization']=f'Bearer {token}'


response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
# print(response.status_code)
# print(response.elapsed)
print(response.content)
print(response.json())

http://52.170.37.14:80/api/v1/service/aks-deploy-test/score
<Response [200]>
b'"politics"'
politics
